<a href="https://colab.research.google.com/github/edermartelinho/Ciencia_de_Dados_Python/blob/main/Ciencia_de_Dados_An%C3%A1lise_ANOVAipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#* Importando as Bibliotecas para as Análises

In [ ]:
!pip install pandas-flavor
!pip install pingouin

In [ ]:
import math
import pandas as pd
import pandas_flavor as pf
import numpy as np
from numpy.testing import suppress_warnings
import numpy.linalg
import numpy.linalg._umath_linalg
#import numpy.linalg._umath_linalg._ilp64
import pingouin as pg
import sklearn
import scipy.stats as sct
from scipy.stats import norm
#from numpy.testing import suppr
from scipy import constants
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import tensorflow as tf

#* Importando o Banco Pelo Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#* Teste Análise de Variância (ANOVA)
Utilizada para comparar a variável de interesse com uma ou mais variáveis categóricas

Pressupostos:
- Variável resposta possui distribuição normal
- Homogeneidade das variâncias (variâncias parecidas)
- Amostras independentes.

In [ ]:
diabeteMellitus=pd.read_csv('/content/drive/MyDrive/Ciencia_Dados_git/ DiabetesMellitus.csv')

In [ ]:
diabeteMellitus['IMC_OMS_CAT'].value_counts()

#* Separando os Grupos

In [ ]:
filtro3  = diabeteMellitus['IMC_OMS_CAT'] == 'Obseso'
filtro4  = diabeteMellitus['IMC_OMS_CAT'] == 'Peso normal'
filtro5  = diabeteMellitus['IMC_OMS_CAT'] == 'Sobrepeso'
obeso = diabeteMellitus[filtro3]
normal = diabeteMellitus[filtro4]
sobrepeso = diabeteMellitus[filtro5]

#* Teste de Homocedasticidade

In [ ]:
[[normal['Pressao Diaastolica'].mean(),normal['Pressao Diaastolica'].std()] , [sobrepeso['Pressao Diaastolica'].mean(), sobrepeso['Pressao Diaastolica'].std()], [obeso['Pressao Diaastolica'].mean(),obeso['Pressao Diaastolica'].std()]]

#* Teste Homogeneidade das Variâncias*
##HO: A variação da pressão diastólica das mulheres com diabete melittus é parecida para qualquer tipo de classificação do IMC (Normal, sobrepeso, obeso)

H1: Pelo menos uma comparação difere

In [ ]:
stats.bartlett(obeso['Pressao Diaastolica'], normal['Pressao Diaastolica'], sobrepeso['Pressao Diaastolica'])

#* Interpretação:
##A variância da pressão diastólica das mulheres com diabette melittus é parecida para a classificação do IMC pelo OMS (p-valor=0,08 > 0,05). É possível também observar através do Boxplot abaixo, que a variabilidade entre os grupos de IMC é parecida apesar de que a variabilidade das mulherss com peso normal ser menor porém não diferente estatisticamente das demais.

In [ ]:
plt.figure(figsize = (10,5))

sns.boxplot(x= 'IMC_OMS_CAT', y = 'Pressao Diaastolica', data = diabeteMellitus, hue='IMC_OMS_CAT', linewidth=4, palette=["m", "g", "r"])
plt.xlabel('Idade')
plt.ylabel('Pressão Diastólica')
plt.title('Boxplot da Pressão Diastólica das Mulheres com Diabete Mellitus Por IMC')

plt.show()

In [ ]:
data=[list(obeso['Pressao Diaastolica']), list(normal['Pressao Diaastolica']), list(sobrepeso['Pressao Diaastolica'])]
pg.homoscedasticity(data,method="bartlett")

In [ ]:
pg.homoscedasticity(data,method="levene")

#* Teste ANOVA*
HO: A pressão diastólica média das mulheres com diabete Mellitus não difere pela Classificação do IMC.

H1: Pelo menos uma média difere.

In [ ]:
ANOVA = diabeteMellitus.anova(dv='Pressao Diaastolica', between=['IMC_OMS_CAT'])
ANOVA

In [ ]:
g = sns.catplot(data=diabeteMellitus, x="IMC_OMS_CAT", y="Pressao Diaastolica", hue="IMC_OMS_CAT",
    capsize=.2, errorbar="se", palette=["m", "g", "r"], kind="point", height=6, aspect=.55,)
g.despine(left=True)#

#*Interpretação: *
##Através do teste ANOVA é possível verificar que a pressão média da presão diastólica das mulheres com diabete Mellitus difere por clasificação do IMC (p-valor=0,00015). É possível observar está diferença no gráfico de médias.

#* Teste de Comparação Múltipla de Tukey
H0: A pressão Diastólica das Mulheres com Diabette Mellitus difere entre mulheres com peso normal e sobrepeso; ou peso normal e obeso; obeso e sobrepeso.

H1: As médias diferem (duas a duas)

In [ ]:
diabeteMellitus.pairwise_tests(dv='Pressao Diaastolica', between=['IMC_OMS_CAT'], effsize='none')


#* Interpretando
##Realizando o teste de comparação múltipla de Tukey, foi verificado que as mulheres com diabete Mellitus classificadas como obesas possuem a pressão diastólica maior que as mulheres com peso normal (p-valor=0,048) e também com as que possuem sobrepeso (p-valor=0.000010), ao nível de 5% de significância. Porém não houve diferença estatísticamente significante da pressão diastólica entre as mulheres com peso normal e sobrepeso. Esses resultados é possível ser observado no Gráfico de Média.



#* Teste Anova Comparando Mais de Uma variável Independente

In [ ]:
ANOVA = diabeteMellitus.anova(dv='Pressao Diaastolica', between=['IMC_OMS_CAT','CatIdade'])
ANOVA

#Perguntas para ser respondida pelo teste:

1) A média da pressão diastólica das mulheres com diabete Mellitus difere pela classificação do IMC?

2) A média da pressão diastólica das mulheres com diabete Mellitus difere pela idade no periodo reprodutivo?

3) A média da pressão diastólica das mulheres com diabete Mellitus difere entre a classificação do IMC juntamente com a idade no período reprodutivo? Ou seja, por exemplo: a pressão diastólica das mulheres obesas difere do período reprodutivo que ela está?



#* Interpretação:
Foi realizado o teste ANOVA para verificar se existia diferença da pressão diastólica entre o IMC e a idade de mulheres. Foi verificado que, existe diferença estatística entre o IMC e a IDADE, poŕem não existe interação entre as variáveis, ao nível de 5% de significância.



#* Teste de Tukey para verificar a diferença entre a média da pressão diastólica das mulheres com idade menor ou igual à 35 anos e mulheres com idade superior à 35 anos.

In [ ]:
diabeteMellitus.pairwise_tests(dv='Pressao Diaastolica', between=('CatIdade'),effsize='none').round(5)

#*Gráficos da Pressão Diastólica Por Idade

In [ ]:
plt.figure(figsize = (10,5))

sns.boxplot(x= 'IMC_OMS_CAT', y = 'Pressao Diaastolica', data = diabeteMellitus, hue='CatIdade', linewidth=4, palette=["m", "g", "r"])
plt.xlabel('Idade')
plt.ylabel('Pressão Diastólica')
plt.title('Boxplot da Pressão Diastólica de Mulheres Por Idade Reprodutiva')

plt.show()

In [ ]:
plt.figure(figsize = (10,5))

sns.catplot(data=diabeteMellitus, x="IMC_OMS_CAT", y="Pressao Diaastolica", hue="IMC_OMS_CAT",col="CatIdade",
    capsize=.2, palette=["m", "g", "r"], errorbar="ci", kind="point", height=6, aspect=.55)
#g.despine(left=True)

plt.xlabel('Idade')
plt.ylabel('Pressão Diastólica')
#plt.title('Boxplot da Pressão Diastólica de Mulheres Por Idade Reprodutiva')

plt.show()

#* Teste Qui-Quadrado de Independência
Utilizado para testar se duas variáveis representadas em uma tabela de contigência são independentes entre si. Ou seja, verificar se as proporções das diversas categorias de uma variável difere nas categorias da outra variável.

Ex.: Verificar se a proporção de mulheres com diabete mellitus diferencia entre mulheres com idade reprodutiva ou não reprodutiva.

#* Construindo as Tabelas que Serão Testadas

In [ ]:
CroosIdadeAbsoluta=pd.crosstab(diabeteMellitus.CatIdade, diabeteMellitus.GlicoseCat, margins=True,normalize="index").mul(100).round(1)
CroosIdadeAbsoluta

#*Teste Qui-Quadrado Utilizando o Scipy

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
expected, observed, stats = pg.chi2_independence(diabeteMellitus, x='CatIdade', y='Diabetes')

In [ ]:
stats.round(15)

In [ ]:
Esperado=expected.round(0)
Esperado

H0: Proporções Parecidas

H1: Pelo Menos 1 Difere

In [ ]:
testequiquadrado= stats
testequiquadrado

#* Interpretação:
Através do teste-quiquadrado de Pearson, foi observado que existe diferença entre as proporções de mulheres entre diabete e idade no período reprodutivo (P-Valor=2.026469e-11). Ou seja, as mulheres com 35 anos ou mais, possuem um porcentagem maior de açucar no sangue (51%) do que as que possuem menos de 35 anos (49%). Já nas mulheres não diabéticas, estas proporções são de 27% e 73%.negrito

#* Análise Gráfica da Correlação

##* Teste de Correlação

#Pressupostos:

Se o teste for o de Pearson, pelo menos uma da variável tem que ter distribuição normal. Já o de spearman, não precisa ter distribuição normal.

HO:  ρ=0,  ou seja, não existe uma correlação importante

H1:  ρ≠0,  a correlação é importante



#* Testando a correlação da Espessura da Dobra Cutânea do Triceps com a pressão diastólica

In [ ]:
diabeteMellitus

In [ ]:
diabeteMellitus['IMC'].corr(diabeteMellitus['Pressao Diaastolica'], method='pearson')

In [ ]:
plt.figure(figsize = [7,5])
plt.scatter( 'IMC', 'Pressao Diaastolica', data=diabeteMellitus)
plt.title('Scatter Plot da Pressão Diastólica em Mulheres com Diabete Mellitus versus A Espessura da Dobra Cutânes Triceps')
plt.ylabel('Pressao Diaastolica')
plt.xlabel('IMC')
plt.show()

In [ ]:
ax = sns.lmplot(x="IMC", y="Pressao Diaastolica", data=diabeteMellitus)
ax.fig.set_size_inches(5, 5)
ax.fig.suptitle('Gráfico Scatter Plot', fontsize=16, y=1.02)
ax.set_xlabels("IMC", fontsize=14)
ax.set_ylabels("Pressão Diastólica", fontsize=14)
ax

In [ ]:
pg.corr(diabeteMellitus['Pressao Diaastolica'], diabeteMellitus['IMC'], method='pearson')

#* Interpretação:
##Através do scatter plot é possível observar que a medida que a espessura da dobra do triceps aumenta, ocorre um leve aumento na pressão diastólica (r=0.25).Através do teste de correlação de Pearson é observado que esta relação é significante ao nível de 5% de significância. Ou seja, a relação existente é significativa e pode ser realizada a regressão linear para criar uma função que descreva essa relação.



#Regressão Linear Simples
A equação da regressão linear simples estimada é dada por:

ŷ=βx+ϵ,

em que: ŷ é o valor da variável resposta ou característica principal;  β  é o quanto y é influenciaado por cada unidade de x;

ϵ  é a influência de outras variáveis que não estão no modelo.

Regressão Linear Múltipla
ŷ=β1x1+β2x2+β3x3+ϵ,

In [ ]:
lmCDiastolica = pg.linear_regression(diabeteMellitus['IMC'],diabeteMellitus['Pressao Diaastolica'])
lmCDiastolica

In [ ]:
#Para um IMC de 30, qual seria o valor da pressão diastólica?
#y=0.76*x+45.89
y=0.76*23+45.89
y

In [ ]:
lmCDiastolica = pg.linear_regression(diabeteMellitus[['IMC','Age', 'concentracao de glicose após duas horas', 'N_gravidez']],diabeteMellitus['Pressao Diaastolica'])
lmCDiastolica

In [ ]:
# Se uma mulher possui 60 anos e um IMC de 32, qual será a pressão diastólica dela?
#y=0.79*x1+0.21*x2+0.02*x3+0.41*x4+33.94
y=0.79*40+0.21*60+0.02*150-0.41*5+33.94
round(y,2)

#* Interpretação:
A relação existente entre a pressão diatólica das mulheres com diabete Mellitus e as variáveis idade, IMC, nº de gravidez e concentração de glicose no sangue após duas horas pode ser descrita pela equação:

y=0.79x1+0.21∗x2+0.41∗x3+0.02∗x4+33.94

Observa-se através da regressão linear múltipla que todos os coeficientes foram signinifantes ao nível de 5% de significância. A cada aumento aumento de uma unidade do IMC, a pressão aumenta 0.79. A cada filho gerado a pressão aumenta 0.41. O intercepto de 33.94, mostra que mais variáveis podem ser acrescentadas no modelo para explicar a pressão diastólica. A variabilidade da pressão diastólica é explicada em  r2=26%,  com outras variáveis no modelo esta variabilidade pode ser mais explciada.



#* Concentração de Glicose no Sangue

In [ ]:
diabeteMellitus['Pressao Diaastolica'].corr(diabeteMellitus['concentracao de glicose após duas horas'],method='spearman')

In [ ]:
pg.corr(diabeteMellitus['Pressao Diaastolica'], diabeteMellitus['concentracao de glicose após duas horas'],method='spearman').round(5)

In [ ]:
plt.figure(figsize = [12,6])
plt.scatter( 'Pressao Diaastolica', 'concentracao de glicose após duas horas', data=diabeteMellitus)
plt.title('Scatter Plot da Concentração de Glicose versus Pressao Diastolica')
plt.ylabel('Concentracao de Glicose após Duas Horas de Jejum')
plt.xlabel('Pressao Diastolica')
plt.show()

#* Insulina no Sangue Após duas Horas de Jejum

In [ ]:
diabeteMellitus['Pressao Diaastolica'].corr(diabeteMellitus['Insulina 2 hrs ((mm U/ml)'],method='spearman')

In [ ]:
TesteCorrelacao=pg.corr(diabeteMellitus['Pressao Diaastolica'], diabeteMellitus['Insulina 2 hrs ((mm U/ml)'],method='spearman').round(5)
TesteCorrelacao

In [ ]:
lmInsulina = pg.linear_regression( diabeteMellitus['Insulina 2 hrs ((mm U/ml)'],diabeteMellitus['Pressao Diaastolica'])
lmInsulina

In [ ]:
plt.figure(figsize = [12,6])
plt.scatter( 'Pressao Diaastolica', 'Insulina 2 hrs ((mm U/ml)', data=diabeteMellitus)
plt.title('Scatter Plot da Quantidade de Insulina no Sangue Após duas Horas de Jejum Versus Pressão Diastólica')
plt.ylabel('Insulina no Sangue Após duas Horas de Jejum')
plt.xlabel('Pressao Diastolica')
plt.show()

#* Número de Gravidez

In [ ]:
diabeteMellitus['Pressao Diaastolica'].corr(diabeteMellitus['N_gravidez'],method='spearman')

In [ ]:
pg.corr(diabeteMellitus['Pressao Diaastolica'], diabeteMellitus['N_gravidez'],method='spearman').round(5)

#* IMC

In [ ]:
diabeteMellitus['Pressao Diaastolica'].corr(diabeteMellitus['IMC'],method='spearman')

In [ ]:
pg.corr(diabeteMellitus['Pressao Diaastolica'], diabeteMellitus['IMC'],method='spearman').round(8)

In [ ]:
lmGravidez = pg.linear_regression(diabeteMellitus['N_gravidez'],diabeteMellitus['Pressao Diaastolica'])
lmGravidez